In [29]:
%cd ..

/content


In [33]:
# !git clone https://github.com/gicait/plitterstreet.git
!pip install --upgrade plitterstreet/.
exit(0)

ERROR: Invalid requirement: 'plitterstreet/.'
Hint: It looks like a path. File 'plitterstreet/.' does not exist.


In [5]:
# download a sample video from https://drive.google.com/file/d/1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0/view?usp=sharing
# ignore downloading and modify path in the below cells, if you already have a video and gps data in csv file

!mkdir /content/plitterstreet-main/demo/data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0" -O /content/plitterstreet-main/demo/data/sample.zip && rm -rf /tmp/cookies.txt
!unzip /content/plitterstreet-main/demo/data/sample.zip -d /content/plitterstreet-main/demo/data/.
!rm -rf '/content/plitterstreet-main/demo/data/sample.zip'

--2022-01-03 18:17:58--  https://docs.google.com/uc?export=download&confirm=Yhko&id=1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0
Resolving docs.google.com (docs.google.com)... 173.194.79.101, 173.194.79.138, 173.194.79.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-a4-docs.googleusercontent.com/docs/securesc/j6l57cht0m04q5lu2632guveo6dbpao4/agrvvg7acltsgt6ck9s75bo3ivs5v4al/1641233850000/10915217646116291607/10021851050897085669Z/1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0?e=download [following]
--2022-01-03 18:17:58--  https://doc-0s-a4-docs.googleusercontent.com/docs/securesc/j6l57cht0m04q5lu2632guveo6dbpao4/agrvvg7acltsgt6ck9s75bo3ivs5v4al/1641233850000/10915217646116291607/10021851050897085669Z/1g5f1eL7iRRW1I99mJwV8ah_le9RH9aY0?e=download
Resolving doc-0s-a4-docs.googleusercontent.com (doc-0s-a4-docs.googleusercontent.com)... 173.194.79.132, 2a00:1450:4013:c05::84
Connecting

In [8]:
from plitterstreet.video import Video

In [10]:
vid = Video('/content/plitterstreet-main/demo/data/sample_video.mp4', '/content/plitterstreet-main/demo/data/sample_GPS5.csv')
vid.ExtractbyGps()

Video is set and ready to make frames
GPS poingts found: 182
destination path is not given, stores the iamges into a created folder with video file name in the directroy to input video
14.0155687 100.6425359 -9.387
0.1187781446813332
100 182
iter: 1, len: 182
0.2827740442101737
182 121
iter: 2, len: 121
0.3003637506109902
121 94
iter: 3, len: 94
0.29743556801352405
94 70
iter: 4, len: 70
0.35633443369095813
70 50
iter: 5, len: 50
0.4382706426974863
50 38
iter: 6, len: 38
0.5664080032664205
38 36
iter: 7, len: 36
0.6648269197324959
36 33
iter: 8, len: 33
0.7632657173040281
33 24
iter: 9, len: 24
0.7801774269648284
24 21
iter: 10, len: 21
0.8172335368195407
21 15
iter: 11, len: 15
0.8207038138068317
15 21
iter: 12, len: 21
0.8207038138068317
21 18
iter: 13, len: 18
0.717993894706219
18 19
iter: 14, len: 19
0.7231210104641048
19 14
iter: 15, len: 14
0.7211818433926005
14 16
iter: 16, len: 16
0.7441124837994335
16 9
iter: 17, len: 9
3.3187941283309996
9 3
iter: 18, len: 3
3.333897299284462

'/content/plitterstreet-main/demo/data/sample_video'

In [5]:
# pip install exif

     |████████████████████████████████| 66 kB 3.4 MB/s 


In [13]:
from plitterstreet.plitter_street import StreetData

In [17]:
street = StreetData('/content/plitterstreet-main/demo/data/sample_video')

In [18]:
street.images_list

'sample_video.csv'

In [19]:
street.load_images_from_path()

# sorted(street.images, key = lambda i: i['name'])

In [10]:
street.images

[{'latitude': 14.0155687,
  'longitude': 100.64253590000001,
  'name': 'sample_video_0.jpg'},
 {'latitude': 14.0155419,
  'longitude': 100.6425214,
  'name': 'sample_video_1.jpg'},
 {'latitude': 14.0155354,
  'longitude': 100.64252560000001,
  'name': 'sample_video_2.jpg'},
 {'latitude': 14.0155007,
  'longitude': 100.6425045,
  'name': 'sample_video_3.jpg'},
 {'latitude': 14.0153873,
  'longitude': 100.6425157,
  'name': 'sample_video_4.jpg'},
 {'latitude': 14.0153673,
  'longitude': 100.64251180000001,
  'name': 'sample_video_5.jpg'},
 {'latitude': 14.0153462,
  'longitude': 100.64251420000001,
  'name': 'sample_video_6.jpg'},
 {'latitude': 14.015339,
  'longitude': 100.64251630000001,
  'name': 'sample_video_7.jpg'},
 {'latitude': 14.0153315,
  'longitude': 100.64251780000001,
  'name': 'sample_video_8.jpg'},
 {'latitude': 14.0153172,
  'longitude': 100.64251850000001,
  'name': 'sample_video_9.jpg'},
 {'latitude': 14.0153026,
  'longitude': 100.64251730000001,
  'name': 'sample_vid

In [20]:
from plitterstreet.predict import EdgePredictor

In [12]:
!rm -rf pLitter/
!git clone https://github.com/gicait/pLitter.git

Cloning into 'pLitter'...
remote: Enumerating objects: 2349, done.
remote: Counting objects: 100% (609/609), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 2349 (delta 449), reused 559 (delta 417), pack-reused 1740
Receiving objects: 100% (2349/2349), 124.67 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (1559/1559), done.


In [22]:
predictor = EdgePredictor('/content/plitterstreet-main/models/')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [23]:
street.predict_on_images(predictor, min_score_thr=0.25)

In [24]:
street.predictions

[{'box': [0.59, 0.57, 0.62, 0.6],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.59},
 {'box': [0.62, 0.75, 0.65, 0.82],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.54},
 {'box': [0.69, 0.87, 0.72, 0.93],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.52},
 {'box': [0.04, 0.58, 0.09, 0.65],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.5},
 {'box': [0.09, 0.76, 0.12, 0.78],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.49},
 {'box': [0.05, 0.31, 0.09, 0.34],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.4},
 {'box': [0.88, 0.94, 0.9, 0.98],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.36},
 {'box': [0.82, 0.3, 0.86, 0.32],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.31},
 {'box': [0.38, 0.81, 0.39, 0.83],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.28},
 {'box': [0.9, 0.67, 0.94, 0.72],

In [25]:
street.save()

In [26]:
import folium
m = folium.Map(tiles="OpenStreetMap")
m = street.draw_street_on_map(m)
m

In [27]:
m.save("/content/plitterstreet-main/demo/data/sample_video.html")